In [ ]:
from sklearn.datasets import fetch_olivetti_faces

In [ ]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import pandas_profiling
import copy
from sklearn.decomposition import PCA
import sklearn.metrics
from sklearn import manifold
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE

In [ ]:
df=fetch_olivetti_faces()

# Narysować wybrane obrazy.

In [ ]:
nrows, ncols = 2, 5
plt.figure(figsize=(10,5))
plt.gray()
for i in range(ncols * nrows):
    ax = plt.subplot(nrows, ncols, i + 1)
    ax.matshow(df.images[i*10,...])
    plt.xticks([]); plt.yticks([])
    plt.title(df.target[i*10])
plt.show()


# Wykorzystać algorytm PCA do kompresji zbioru Olivetti Faces. 

In [ ]:
from sklearn.decomposition import PCA
pca = PCA().fit(df.data)

plt.figure(figsize=(9,6))
plt.plot(range(1, len(pca.explained_variance_ratio_)+1), np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

Wybieramy takie `n_components` aby `cumulative explained variance` było ok, 0.95

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 0.99)
pca.fit(df.data)
reduced = pca.transform(df.data)


Policzmy stopień kompresji

In [ ]:
print(f"CR = {df.data.shape[1]/reduced.shape[1]}")

In [ ]:
re_reduced = pca.inverse_transform(reduced)

In [ ]:
nrows, ncols = 2, 5
plt.figure(figsize=(10,5))
plt.gray()
for i in range(ncols * nrows):
    ax = plt.subplot(nrows, ncols, i + 1)
    ax.matshow(re_reduced[i*10].reshape((64,64)))
    plt.xticks([]); plt.yticks([])
    plt.title(i)
plt.show()

Po preprowadzeniu transformacji odwrotnej można zaóważyć spadek jakości względem pierwotnych obrazów

In [ ]:
from sklearn.metrics import mean_squared_error
for i in range(10):
    print(f'RMSE obraz {i}: {np.sqrt(mean_squared_error(df.data[i*10], re_reduced[i*10])):.3f}')

# Modyfikacje
## rotacja

In [ ]:
rotated = []
for image in df.images:
    rotated.append(np.transpose(image))
rotated = np.array(rotated)

In [ ]:
nrows, ncols = 2, 5
plt.figure(figsize=(10,5))
plt.gray()
for i in range(ncols * nrows):
    ax = plt.subplot(nrows, ncols, i + 1)
    ax.matshow(rotated[i*10,...])
    plt.xticks([]); plt.yticks([])
    plt.title(df.target[i*10])
plt.show()


## odbicie lustrzane

In [ ]:
mirror = []
for image in df.images:
    mirror.append(np.fliplr(image))
mirror = np.array(mirror)

In [ ]:
nrows, ncols = 2, 5
plt.figure(figsize=(10,5))
plt.gray()
for i in range(ncols * nrows):
    ax = plt.subplot(nrows, ncols, i + 1)
    ax.matshow(mirror[i*10,...])
    plt.xticks([]); plt.yticks([])
    plt.title(df.target[i*10])
plt.show()

## przyciemnione

In [ ]:
dark = []
for image in df.data:
    dark.append((image - 0.5).clip(min=0))
dark = np.array(dark)

In [ ]:
nrows, ncols = 2, 5
plt.figure(figsize=(10,5))
plt.gray()
for i in range(ncols * nrows):
    ax = plt.subplot(nrows, ncols, i + 1)
    ax.matshow(dark[i*10,...].reshape((64,64)))
    plt.xticks([]); plt.yticks([])
    plt.title(df.target[i*10])
plt.show()

# Transformacja modyfikacji

In [ ]:
reduced = pca.transform(df.data)
re_reduced = pca.inverse_transform(reduced)

In [ ]:
for i in range(10):
    print(f'RMSE obraz {i}: {np.sqrt(mean_squared_error(df.data[i*10], re_reduced[i*10])):.3f}')

## rotacja

In [ ]:
reduced = pca.transform(rotated.reshape((400,4096)))
re_reduced = pca.inverse_transform(reduced)

In [ ]:
for i in range(10):
    print(f'RMSE obraz {i} rotated: {np.sqrt(mean_squared_error(rotated.reshape((400,4096))[i*10], re_reduced[i*10])):.3f}')

Wartosci błędu ok. 7 razy większa niż dla orginalnych obrazów

## odbicie lustrzane

In [ ]:
reduced = pca.transform(mirror.reshape((400,4096)))
re_reduced = pca.inverse_transform(reduced)

In [ ]:
for i in range(10):
    print(f'RMSE obraz {i} mirrored: {np.sqrt(mean_squared_error(mirror.reshape((400,4096))[i*10], re_reduced[i*10])):.3f}')

Odbcia lustrzane mają największe RMSE dla obrazów które orginalnie miały najmniejsze. RMSE nie jest tak duże jak dla rotacji.

## przyciemnione

In [ ]:
reduced = pca.transform(dark.reshape((400,4096)))
re_reduced = pca.inverse_transform(reduced)

In [ ]:
for i in range(10):
    print(f'RMSE obraz {i} darken: {np.sqrt(mean_squared_error(dark.reshape((400,4096))[i*10], re_reduced[i*10])):.3f}')

Wyniki RMSE dla przyciemnienia są stabilne `(0.03x)` podobnie jak dla oryginalych obrazów `(0.01x)`

# Czy PCA może służyć do wykrywania pewnego typu anomalii w zdjęciach twarzy?

Wydaje mi się że tak. RMSE po PCA zdjęć zmodyfikowanych było często największe dla zdjęć które pierwotnie miały najmniejsze RMSE. Może to oznaczać że te obrazy miały jakieś elementy wyróżniające się które pozwalały łatwiej odtworzyć ten obraz po kompresji, ale model nie radził sobię z przywracaniem tych cech szczególnych po modyfikacji obrazu. Jednym z takich obrazów był ten pokazany niżej. Może ma to związek z krzywym nosem lub ustami.

In [ ]:
plt.figure(figsize=(10,5))
plt.gray()
plt.matshow(re_reduced[1*10].reshape((64,64)))
plt.xticks([]); plt.yticks([])
plt.title(1)
plt.show()